# Artificial Intelligence - Laboratory 05

## _Searching algorithms for optimal decision-making in game theory and AI_


## Introduction

In gaming tehory, the _decision-making process_ relies on the searching algorithm guiding the investigation of the search-space.

Today's challenge sets the **MinMax Algorithm** as the main character of a  two-player game.

Using tic-tac-toe as an example, the algorithm should compute the next best move by evaluating the utility of the board.

For this problem the utility value can be:

* _-1_ if player that seeks minimum wins;
* _0_ if it's a tie;
* _1_ if player that seeks maximum wins.



In [377]:
board = [" " for _ in range(9)]

In [378]:
def display_board(board):
    print(f"\n{'-'*20}\n")
    for i in range(0, 9, 3):
        print(" | ".join(board[i:i+3]))
        if i < 6:
            print("-" * 9)
display_board(board)


--------------------

  |   |  
---------
  |   |  
---------
  |   |  


In [379]:
# the method should return True or False based on who won the game
def check_win(board, player):

    winning_lines = [(0,1,2), (3,4,5), (6,7,8), 
                     (0,3,6), (1,4,7), (2,5,8), 
                     (0,4,8), (2,4,8)
                     ]
    

    return any(all(board[i] == player for i in line) for line in  winning_lines)


In [380]:
# Function to check for a draw
def check_draw(board):
    return check_win(board, "X") == check_win(board, "O") == False and all(cell != " " for cell in board)

## Min-Max Algorithm


Build the search game tree to determine the best move using:

* the AI's strategy is to _maximise_ its score while the opponent's score minimises;
* the human's strategy is to _minimise_ AI's score.

In [381]:
# Function for the minimax algorithm
def minimax(board, depth, maximizing_player):
    # Base cases: check for terminal states
    if check_win(board, "O"): # AI Wins
        return 10 - depth # Higher estimates will be chosen so good endings will come sooner
    if check_win(board, "X"): # Human Wins
        return -10 + depth # Least negative  estimates will be chosen so bad endings are delayed
    if check_draw(board):
        return 0

    if maximizing_player:
        # AI Strategy
        best_eval = float("-inf")
        for i in range (9):
            if board[i] == " ":
                board[i] = "O"
                eval_i = minimax(board, depth + 1, False)
                board[i] = " "
                best_eval = max(best_eval, eval_i)
        return best_eval
        
    else:
        # Human Strategy
        best_eval = float("inf")
        for i in range (9):
            if board[i] == " ":
                board[i] = "X"
                eval_i = minimax(board, depth + 1, True)
                board[i] = " "
                best_eval = max(best_eval, eval_i)
        return best_eval

Improve your algorithm with alpha-beta pruning 

In [382]:
def minimaxAB(board, depth, maximizing_player, alpha, beta):
    # Terminal scoring (prefer faster wins, delay losses)
    if check_win(board, "O"):
        return 10 - depth
    if check_win(board, "X"):
        return -10 + depth
    if check_draw(board):
        return 0

    if maximizing_player:  # AI ("O")
        value = -10_000
        for i in range(9):
            if board[i] == " ":
                board[i] = "O"
                value = max(value, minimaxAB(board, depth + 1, False, alpha, beta))
                board[i] = " "
                alpha = max(alpha, value)
                if beta <= alpha:  # prune
                    break
        return value
    else:  # Human ("X")
        value = 10_000
        for i in range(9):
            if board[i] == " ":
                board[i] = "X"
                value = min(value, minimaxAB(board, depth + 1, True, alpha, beta))
                board[i] = " "
                beta = min(beta, value)
                if beta <= alpha:  # prune
                    break
        return value


In [383]:
# Function to find the best move using the minimax algorithm
import math
def best_move(board, depth):
    best_eval = float("-inf")
    best_move = -1
    for i in range(9):
        if board[i] == " ":
            board[i] = "O"
            evaluation = minimaxAB(board, depth, False, -float("inf"), float("inf")) #or minimax(board, 0, False, -math.inf, math.inf)
            board[i] = " "
            if evaluation > best_eval:
                best_eval = evaluation
                best_move = i
    return best_move

In [384]:
import random

def select_next_move(board, random_player: bool):
   
    empties = [i for i, c in enumerate(board) if c == " "]
    if not empties:
        return -1

    if random_player:
        return random.choice(empties)

    while True:
        try:
            i = int(input("Enter your move (0-8): "))
            if i in empties:
                return i
            print("Invalid move. Pick an empty cell 0–8 that isn’t taken.")
        except ValueError:
            print("Please enter a number 0–8.")


In [385]:
def play_game(randomPlayer: bool, depth):
    while True:
        display_board(board)

        player_move = select_next_move(board, randomPlayer)
        
        if board[player_move] != " ":
            print("Invalid move. Try again.")
            continue
        
        board[player_move] = "X"
        
        if check_win(board, "X"):
            display_board(board)
            print("You win!")
            break
        
        ai_move = best_move(board, depth)
        board[ai_move] = "O"
        
        if check_win(board, "O"):
            display_board(board)
            print("AI wins!")
            break
        
        if check_draw(board):
            display_board(board)
            print("It's a draw!")
            break

Change the human player with a random player

In [386]:
# play_game(False, 0)

# play_game(True, 0)


Modify the minimax method so that it uses the depth parameter. Test how a depth of 3 compares to a player than can make moves until the game is over.

In [387]:
play_game(True, 5)


--------------------

  |   |  
---------
  |   |  
---------
  |   |  

--------------------

O |   |  
---------
  | X |  
---------
  |   |  

--------------------

O |   | O
---------
  | X |  
---------
  |   | X

--------------------

O | O | O
---------
  | X |  
---------
X |   | X
AI wins!
